In [ ]:
#https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction
import pandas as pd
df = pd.read_csv('heart.csv')

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools
from tqdm import tqdm
import tensorflow_datasets as tfds

In [ ]:
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [ ]:
df.isna().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

In [ ]:
df.dtypes

Age                 int64
Sex                object
ChestPainType      object
RestingBP           int64
Cholesterol         int64
FastingBS           int64
RestingECG         object
MaxHR               int64
ExerciseAngina     object
Oldpeak           float64
ST_Slope           object
HeartDisease        int64
dtype: object

In [ ]:
for col in df.columns:
  if df[col].dtype == 'object':
    df[col] = df[col].astype('category').cat.codes

In [ ]:
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,1,1,140,289,0,1,172,0,0.0,2,0
1,49,0,2,160,180,0,1,156,0,1.0,1,1
2,37,1,1,130,283,0,2,98,0,0.0,2,0
3,48,0,0,138,214,0,1,108,1,1.5,1,1
4,54,1,2,150,195,0,1,122,0,0.0,2,0


In [ ]:
X = df.drop(columns = 'HeartDisease')
y = df['HeartDisease']

In [ ]:
train, test = train_test_split(df, test_size=0.4)

In [ ]:
len(train)

550

In [ ]:
len(test)

368

In [ ]:
230/43

5.348837209302325

In [ ]:
train_stats = train.describe()
train_stats.pop('HeartDisease')
train_stats = train_stats.transpose()

In [ ]:
train_stats

,count,mean,std,min,25%,50%,75%,max
Age,550.0,53.745455,9.408980,29.0,47.00,54.0,60.00,77.0
Sex,550.0,0.787273,0.409609,0.0,1.00,1.0,1.00,1.0
ChestPainType,550.0,0.790909,0.960945,0.0,0.00,0.0,2.00,3.0
RestingBP,550.0,132.723636,18.510372,80.0,120.00,130.0,140.00,200.0
Cholesterol,550.0,195.909091,111.480919,0.0,168.25,221.0,263.75,603.0
FastingBS,550.0,0.229091,0.420630,0.0,0.00,0.0,0.00,1.0
RestingECG,550.0,0.990909,0.608013,0.0,1.00,1.0,1.00,2.0
MaxHR,550.0,135.909091,25.235339,60.0,120.00,137.0,154.00,202.0
ExerciseAngina,550.0,0.410909,0.492447,0.0,0.00,0.0,1.00,1.0
Oldpeak,550.0,0.909273,1.061757,-2.6,0.00,0.7,1.50,5.0


In [ ]:
y_train = train.pop('HeartDisease')
y_test = test.pop('HeartDisease')

In [ ]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']

In [ ]:
X_train_norm = norm(train)
X_test_norm = norm(test)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_norm.values, y_train.values))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test_norm.values, y_test.values))

In [ ]:
list(enumerate(train_dataset))[:2]

[(0,
  (<tf.Tensor: shape=(11,), dtype=float64, numpy=
   array([ 0.02437827,  0.52216043,  1.29144703, -0.72909007,  0.32737374,
          -0.53060198,  0.04264014,  0.53269164,  1.20075734,  0.64900487,
          -0.63078485])>,
   <tf.Tensor: shape=(), dtype=int64, numpy=1>)),
 (1,
  (<tf.Tensor: shape=(11,), dtype=float64, numpy=
   array([ 1.05576682,  0.52216043, -0.80382101,  0.64282567, -1.88023969,
          -0.53060198,  1.60611206, -0.56037315,  1.20075734,  0.15474749,
          -0.63078485])>,
   <tf.Tensor: shape=(), dtype=int64, numpy=1>))]

In [ ]:
batch_size = 32
train_dataset = train_dataset.shuffle(buffer_size = len(train)).batch(batch_size).prefetch(1)
test_dataset = test_dataset.batch(batch_size=batch_size).prefetch(1)

In [ ]:
def base_model():
  inputs = tf.keras.layers.Input(shape = len(train.columns))
  x = tf.keras.layers.Dense(64, activation='LeakyReLU')(inputs)
  x = tf.keras.layers.Dense(64, activation='LeakyReLU')(x)
  outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

  model = tf.keras.Model(inputs=inputs, outputs = outputs)
  return model

model = base_model()

In [ ]:
optimizer = tf.keras.optimizers.legacy.RMSprop(learning_rate=0.001)
loss_object = tf.keras.losses.BinaryCrossentropy()

In [ ]:
outputs = model(X_test_norm.values)
loss = loss_object(y_true = np.expand_dims(y_test, axis = -1), y_pred=outputs)
loss.numpy()

0.6951292

In [ ]:
def gradient_calc(optimizer, loss_object, model, X, y):
  with tf.GradientTape() as tape:
    logits = model(X)
    loss = loss_object(y_true=y, y_pred=logits)

  gradients = tape.gradient(loss, model.trainable_weights)
  optimizer.apply_gradients(zip(gradients, model.trainable_weights))

  return logits, loss

In [ ]:
def training_one_epoch(train_data, optimizer, loss_object, model):
  losses = []
  for step, (x_batch, y_batch) in enumerate(train_data):
    logits, loss = gradient_calc(optimizer, loss_object, model, x_batch, y_batch)
    losses.append(loss)

    logits = tf.round(logits)
    logits = tf.cast(logits, 'int64')

    train_acc_metric.update_state(y_batch, logits)
  return losses

In [ ]:
def validation_loss():
  losses = []
  for val_x, val_y in test_dataset:
    val_logits = model(val_x)
    val_loss = loss_object(y_true = val_y, y_pred=val_logits)
    losses.append(val_loss)

    val_logits = tf.cast(tf.round(model(val_x)), 'int64')

    val_acc_metric.update_state(val_y, val_logits)
  return losses

In [ ]:
train_acc_metric = tf.keras.metrics.BinaryAccuracy()
val_acc_metric = tf.keras.metrics.BinaryAccuracy()

In [ ]:
epochs = 60
val_losses, train_losses = [], []

for epoch in range(epochs):
  print('Start of epoch %d' % (epoch,))

  train_loss = training_one_epoch(train_dataset, optimizer, loss_object, model)

  train_acc = train_acc_metric.result()
  val_loss = validation_loss()

  val_acc = val_acc_metric.result()

  train_losses_mean = np.mean(train_loss)
  val_losses_mean = np.mean(val_loss)

  val_losses.append(val_losses_mean)
  train_losses.append(train_losses_mean)

  print('\n Epcoh %s: Training loss: %.3f  Validation Loss: %.3f, Training Accuracy: %.3f, Validation Accuracy %.3f' % (epoch, float(train_losses_mean), float(val_losses_mean), float(train_acc), float(val_acc)))

  train_acc_metric.reset_states()
  val_acc_metric.reset_states()

Start of epoch 0

 Epcoh 0: Training loss: 0.460  Validation Loss: 0.409, Training Accuracy: 0.818, Validation Accuracy 0.834
Start of epoch 1

 Epcoh 1: Training loss: 0.363  Validation Loss: 0.388, Training Accuracy: 0.849, Validation Accuracy 0.842
Start of epoch 2

 Epcoh 2: Training loss: 0.335  Validation Loss: 0.385, Training Accuracy: 0.853, Validation Accuracy 0.859
Start of epoch 3

 Epcoh 3: Training loss: 0.325  Validation Loss: 0.385, Training Accuracy: 0.856, Validation Accuracy 0.848
Start of epoch 4

 Epcoh 4: Training loss: 0.332  Validation Loss: 0.383, Training Accuracy: 0.855, Validation Accuracy 0.856
Start of epoch 5

 Epcoh 5: Training loss: 0.336  Validation Loss: 0.383, Training Accuracy: 0.865, Validation Accuracy 0.848
Start of epoch 6

 Epcoh 6: Training loss: 0.330  Validation Loss: 0.381, Training Accuracy: 0.862, Validation Accuracy 0.851
Start of epoch 7

 Epcoh 7: Training loss: 0.330  Validation Loss: 0.380, Training Accuracy: 0.865, Validation Accurac

In [ ]:
len(train)

642

In [ ]:
len(test)

276

In [ ]:
276/32

8.625

In [ ]:
32*8

256

In [ ]:
276-256

20